# 09: Statistical Tests for Model Comparison

**Diebold-Mariano, Pesaran-Timmermann, and HAC Variance**

---

## 🚨 If You Know sklearn But Not Forecast Comparison, Read This First

**What you already know (from standard ML)**:
- Compare models by error: "Model A has lower MAE than Model B"
- Use t-test: `ttest_rel(errors_A, errors_B)` to check significance
- p < 0.05 means the difference is real
- Done!

**What's different with time series**:

| Standard ML | Time Series Reality |
|-------------|---------------------|
| "Model A: MAE=0.08, Model B: MAE=0.09" | Sampling variability may reverse this |
| `ttest_rel` for significance | **Invalid**: assumes independent errors |
| Independent errors | Errors are **correlated** (MA(h-1) structure) |
| Standard SE = σ/√n | Underestimates variance → false significance |

**The problem in practice**:

```python
# You ran this:
from scipy.stats import ttest_rel
stat, p = ttest_rel(model_A_errors, model_B_errors)
# p = 0.02 → "Model A is significantly better!"

# Reality:
# Errors are autocorrelated → SE is too small → p is inflated
# Proper test: p = 0.15 → "No significant difference"
```

**The fix**: Use the Diebold-Mariano test with HAC variance.

---

## What You'll Learn

1. **Why MAE comparison isn't enough** — Sampling variability and autocorrelation invalidate naive comparisons
2. **Diebold-Mariano test** — Rigorous statistical test for comparing forecast accuracy [T1]
3. **Pesaran-Timmermann test** — Testing directional accuracy significance [T1]

**Prerequisites**: Notebooks 01-04 (Foundation tier)

---

## The Problem: "My Model Has Lower MAE" Isn't Enough

Consider this common scenario:

```
Model A MAE: 0.0823
Model B MAE: 0.0891
Conclusion: Model A is better!
```

**Why this reasoning fails:**

1. **Sampling variability** — With different test data, Model B might win
2. **Autocorrelated errors** — Standard t-tests assume independent errors (violated in time series)
3. **Multiple comparisons** — Testing 5 models inflates false positive rate

**The solution**: Statistical tests designed for forecast comparison.

In [ ]:
import numpy as np
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

from temporalcv.statistical_tests import (
    dm_test,
    pt_test,
    compare_multiple_models,
)
from temporalcv.cv import WalkForwardCV

np.random.seed(42)
print("temporalcv statistical tests")

In [ ]:
# Generate synthetic data
def generate_ar1(n: int, phi: float = 0.9, sigma: float = 0.1) -> np.ndarray:
    """Generate AR(1) process: y[t] = phi * y[t-1] + epsilon[t]"""
    y = np.zeros(n)
    y[0] = np.random.normal(0, sigma)
    for t in range(1, n):
        y[t] = phi * y[t - 1] + np.random.normal(0, sigma)
    return y

# Create dataset
n = 500
y = generate_ar1(n, phi=0.9)

# Create features (lagged values)
X = np.column_stack([y[:-2], y[1:-1]])
y_target = y[2:]

# Train-test split (temporal)
train_size = int(len(y_target) * 0.7)
X_train, y_train = X[:train_size], y_target[:train_size]
X_test, y_test = X[train_size:], y_target[train_size:]

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

In [ ]:
# Train multiple models
models = {
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.01),
    "RF": RandomForestRegressor(n_estimators=50, max_depth=3, random_state=42),
}

predictions = {}
errors = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    predictions[name] = pred
    errors[name] = y_test - pred  # Error = actual - prediction
    mae = np.mean(np.abs(errors[name]))
    print(f"{name} MAE: {mae:.4f}")

# Add persistence baseline
persistence_pred = X_test[:, -1]  # y[t-1] as prediction for y[t]
predictions["Persistence"] = persistence_pred
errors["Persistence"] = y_test - persistence_pred
print(f"Persistence MAE: {np.mean(np.abs(errors['Persistence'])):.4f}")

**Naive conclusion**: The model with lowest MAE is "best".

**But is this difference statistically significant?**

---

## 1. Diebold-Mariano Test [T1]

The DM test compares the predictive accuracy of two forecasts.

**Null hypothesis**: Both forecasts have equal expected loss.

**Key features:**
- Handles autocorrelated forecast errors (via HAC variance)
- Works with any loss function (squared, absolute)
- Harvey correction for small samples [T1]

**Citation**: Diebold & Mariano (1995), Harvey et al. (1997)

In [ ]:
# Compare Ridge vs Persistence
result = dm_test(
    errors["Ridge"],
    errors["Persistence"],
    h=1,  # 1-step forecast
    loss="squared",  # MSE loss
    harvey_correction=True,  # Small-sample adjustment
)

print(f"DM statistic: {result.statistic:.3f}")
print(f"P-value: {result.pvalue:.4f}")
print(f"Mean loss differential: {result.mean_loss_diff:.6f}")
print(f"Significant at 5%? {result.pvalue < 0.05}")

### Interpreting DM Results

- **Negative statistic**: Model 1 (Ridge) has lower loss → Ridge is better
- **Positive statistic**: Model 2 (Persistence) has lower loss → Persistence is better
- **p < 0.05**: Reject null hypothesis — difference is significant

In [ ]:
# DMTestResult has convenient properties
print(f"\nResult summary:")
print(result)

### HAC Variance for h-step Forecasts

For h-step ahead forecasts, errors follow an MA(h-1) structure:
- 1-step: Errors are uncorrelated (standard variance)
- 2-step: Errors have lag-1 autocorrelation
- h-step: Errors have autocorrelation up to lag h-1

**HAC (Heteroskedasticity and Autocorrelation Consistent) variance** accounts for this.

In [ ]:
# Compare 1-step vs 2-step horizon
print("Effect of horizon on DM test:")
print("="*50)

for h in [1, 2, 3]:
    result = dm_test(
        errors["Ridge"],
        errors["Persistence"],
        h=h,
        loss="squared",
    )
    print(f"h={h}: DM={result.statistic:6.3f}, p={result.pvalue:.4f}")

**Key insight**: Higher horizon → wider confidence intervals → harder to reject null.

This is correct behavior! More uncertainty in h-step forecasts.

---

## 2. Pesaran-Timmermann Test [T1]

The PT test evaluates **directional accuracy** — did you predict the right sign?

**Null hypothesis**: Forecasts have no directional accuracy (random guessing).

**Key features:**
- Compares observed accuracy to expected under independence
- Accounts for marginal distributions (not just 50%)
- Works on changes/returns (not levels)

**Citation**: Pesaran & Timmermann (1992)

In [ ]:
# Compute changes (PT test works on changes, not levels)
actual_changes = np.diff(y_test)
ridge_changes = np.diff(predictions["Ridge"])

# Run PT test
result = pt_test(actual_changes, ridge_changes)

print(f"Observed accuracy: {result.accuracy:.1%}")
print(f"Expected accuracy: {result.expected:.1%}")
print(f"PT statistic: {result.statistic:.3f}")
print(f"P-value: {result.pvalue:.4f}")
print(f"Significant? {result.pvalue < 0.05}")

### Why Expected Accuracy ≠ 50%

The expected accuracy under independence depends on marginal distributions:

```
Expected = P(actual > 0) × P(pred > 0) + P(actual < 0) × P(pred < 0)
```

If actuals are 60% positive and predictions are 70% positive:
- Expected = 0.6 × 0.7 + 0.4 × 0.3 = 0.54 (not 0.50)

In [ ]:
# Test persistence baseline direction accuracy
persistence_changes = np.diff(predictions["Persistence"])

result_persistence = pt_test(actual_changes, persistence_changes)
print(f"\nPersistence direction accuracy:")
print(f"Observed: {result_persistence.accuracy:.1%}")
print(f"Expected: {result_persistence.expected:.1%}")
print(f"P-value: {result_persistence.pvalue:.4f}")

### 3-Class PT Test [T3]

The standard PT test uses 2 classes (UP/DOWN). With a threshold, we can use 3 classes:

- **UP**: change > threshold
- **DOWN**: change < -threshold
- **FLAT**: |change| ≤ threshold

**Note**: 3-class mode is [T3] — an exploratory extension, not academically validated.

In [ ]:
# 3-class PT test with move threshold
move_threshold = np.percentile(np.abs(actual_changes), 70)

result_3class = pt_test(
    actual_changes, 
    ridge_changes,
    move_threshold=move_threshold  # Enables 3-class mode
)

print(f"3-class PT test (threshold={move_threshold:.4f}):")
print(f"Observed accuracy: {result_3class.accuracy:.1%}")
print(f"Expected accuracy: {result_3class.expected:.1%}")
print(f"P-value: {result_3class.pvalue:.4f}")
print(f"Number of classes: {result_3class.n_classes}")

---

## 3. Multi-Model Comparison

When comparing multiple models, we need:
- All pairwise DM tests
- **Bonferroni correction** for multiple comparisons

**Without correction**: 5 models → 10 pairs → 40% false positive rate at α=0.05

In [ ]:
# Compare all models
comparison = compare_multiple_models(
    errors,  # Dict of model_name -> error_array
    h=1,
    alpha=0.05,
    loss="squared",
)

print(comparison.summary())

In [ ]:
# Access specific pairwise results
print(f"\nBest model: {comparison.best_model}")
print(f"\nBonferroni-corrected alpha: {comparison.bonferroni_alpha:.4f}")
print(f"\nSignificant pairs (after correction): {comparison.significant_pairs}")

In [ ]:
# Get specific pairwise result
ridge_vs_lasso = comparison.get_pairwise("Ridge", "Lasso")
if ridge_vs_lasso:
    print(f"Ridge vs Lasso:")
    print(f"  DM statistic: {ridge_vs_lasso.statistic:.3f}")
    print(f"  P-value: {ridge_vs_lasso.pvalue:.4f}")

---

## Pitfall Section: Common Mistakes

### Pitfall 1: Wrong Horizon Parameter

```python
# WRONG: Using h=1 for 3-step ahead forecasts
result = dm_test(errors_3step, baseline_errors, h=1)  # Underestimates variance!

# RIGHT: Match h to actual forecast horizon
result = dm_test(errors_3step, baseline_errors, h=3)  # Correct HAC bandwidth
```

**Why it matters**: Wrong h → incorrect variance estimate → invalid p-values.

---

### Pitfall 2: Ignoring Harvey Correction

```python
# WRONG: Disable Harvey correction with small samples
result = dm_test(errors_a, errors_b, harvey_correction=False)  # n < 100

# RIGHT: Always use Harvey correction (default)
result = dm_test(errors_a, errors_b, harvey_correction=True)
```

**Why it matters**: Without correction, DM test has size distortions in small samples.

---

### Pitfall 3: Too Few Observations

```python
# WRONG: Run DM test with n < 30
result = dm_test(errors[:20], baseline[:20])  # ValueError!

# RIGHT: Ensure n >= 30
if len(errors) >= 30:
    result = dm_test(errors, baseline)
```

**Why it matters**: Asymptotic approximation breaks down with few observations.

In [ ]:
# Demonstrate minimum sample size requirement
try:
    small_errors = errors["Ridge"][:20]
    small_baseline = errors["Persistence"][:20]
    dm_test(small_errors, small_baseline)
except ValueError as e:
    print(f"Expected error: {e}")

---

## Key Insights

```
★ Insight ─────────────────────────────────────

1. MAE comparison alone is insufficient
   - Use DM test for statistical significance
   - Sampling variability can fool you

2. DM test accounts for autocorrelated errors
   - HAC variance for h-step forecasts
   - Harvey correction for small samples [T1]

3. PT test evaluates directional accuracy
   - Expected accuracy ≠ 50% (depends on marginals)
   - 3-class mode is [T3] exploratory

4. Multiple comparisons need correction
   - Bonferroni: α_corrected = α / n_comparisons
   - 5 models → 10 pairs → significant inflation

───────────────────────────────────────────────
```

---

## Next Steps

| Notebook | Topic |
|----------|-------|
| **10** | High-persistence metrics (MC-SS, Theil's U) |
| 11 | Conformal prediction for uncertainty quantification |
| 12 | Regime-stratified evaluation (capstone) |

---

**You've learned**: How to rigorously compare forecasting models using DM and PT tests with proper variance estimation and multiple comparison corrections.